Identify Journal Oids
===

Identify ids of journals needed for text feature analysis.

Looks at the generated train and test metadata to generate a list of journal oids.

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
import os
import json
import sys
import pickle
from tqdm import tqdm

import dateutil.parser
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta
import pytz

In [ ]:
from pathlib import Path
git_root_dir = !git rev-parse --show-toplevel
git_root_dir = Path(git_root_dir[0].strip())
git_root_dir

In [ ]:
import sys
sys.path.append(os.path.join(git_root_dir, 'src'))
import cbrec.genconfig

In [ ]:
config = cbrec.genconfig.Config()
#config.metadata_filepath += "_old"
#config.feature_db_filepath += "_old"

In [ ]:
import cbrec.featuredb
import cbrec.utils
import cbrec.reccontext

In [ ]:
md_list = cbrec.utils.get_metadata_list(config.metadata_filepath)
len(md_list)

In [ ]:
df = cbrec.utils.create_metadata_dataframe(md_list)
len(df)

In [ ]:
pd.DataFrame(df.type.value_counts().rename("Total metadata count by type"))

In [ ]:
# filter the md_list
md_list = [md for md in md_list if md['type'] != 'ineligible']
len(md_list)

In [ ]:
# put all derived data in the data_selection folder
working_dir = "/home/lana/shared/caringbridge/data/projects/recsys-peer-match/model_data"
os.makedirs(working_dir, exist_ok=True)
working_dir

In [ ]:
md_type = 'train'
journal_oid_set = set()

for md in md_list:
    if md['type'] != md_type:
        continue
    for usp_type in ['source_usp', 'target_usp', 'alt_usp']:
        journals_key = usp_type + "_recent_journals"
        if journals_key in md:
            journals = md[journals_key]
            for usp, recent_journal_ids in journals.items():
                for journal_oid in recent_journal_ids:
                    journal_oid_set.add(journal_oid)

journal_oids_filepath = os.path.join(working_dir, f'{md_type}_journal_oids.txt')
with open(journal_oids_filepath, 'w') as outfile:
    written_count = 0
    for journal_oid in journal_oid_set:
                        outfile.write(journal_oid + "\n")
                        written_count += 1
written_count

In [ ]:
list(journal_oid_set)[:10]